# Load

In [1]:
import io
import logging
import math
import sys
import pandas as pd
from typing import Tuple, get_args
from py_ballisticcalc import Ammo, Atmo, Vacuum, Shot, Calculator, HitResult
from py_ballisticcalc import ZeroFindingError, RangeError, TrajFlag, BaseEngineConfigDict, SciPyEngineConfigDict
from py_ballisticcalc import TableG1, logger
from py_ballisticcalc.drag_model import DragModel
from py_ballisticcalc.helpers import must_fire, find_index_for_time_point
from py_ballisticcalc.unit import *
from py_ballisticcalc.interface import _EngineLoader
logger.setLevel(logging.DEBUG)
print("\nAvailable engines: " + str(sorted([e.name for e in _EngineLoader.iter_engines()])))
PreferredUnits.drop = Distance.Feet
PreferredUnits.distance = Distance.Feet


Available engines: ['cythonized_euler_engine', 'cythonized_rk4_engine', 'euler_engine', 'rk4_engine', 'scipy_engine', 'verlet_engine']


# Vacuum Trajectory

Here is a convenient vacuum scenario so that we can check engine precision against a known exact solution.

In [2]:
from py_ballisticcalc.helpers import solve_velocity_for_vacuum_time_to_zero, vacuum_time_to_zero, vacuum_range
cGravityConstant: float = -32.17405  # feet per second squared
launch_angle_deg = 30.0  # degrees
launch_angle_rad = math.radians(launch_angle_deg)
time_to_zero = 60.0  # seconds
launch_velocity_fps = solve_velocity_for_vacuum_time_to_zero(time_to_zero=time_to_zero, launch_angle_deg=launch_angle_deg, gravity=-cGravityConstant)
time = vacuum_time_to_zero(velocity=launch_velocity_fps, launch_angle_deg=launch_angle_deg, gravity=-cGravityConstant)
range_to_zero = vacuum_range(velocity=launch_velocity_fps, angle_in_degrees=launch_angle_deg, gravity=-cGravityConstant)
print(f'{time} seconds to return to zero, launching at {launch_velocity_fps}fps with {launch_angle_deg} degrees.  '
      f'Range={range_to_zero:.4f}ft.')

def check_error(hit: HitResult, output: bool = False) -> float:
    z_down = hit.flag(TrajFlag.ZERO_DOWN)
    if z_down is not None and output:
        print(f'ZERO_DOWN: {z_down.time}s to ({z_down.distance >> Distance.Feet}, {z_down.height >> Distance.Feet}).')
    chkpt = hit.get_at_distance(Distance.Feet(range_to_zero))
    if chkpt is not None:
        chk_x = chkpt.distance >> Distance.Feet
        chk_h = chkpt.height >> Distance.Feet
        chk_err = math.sqrt((chk_x-range_to_zero)**2 + chk_h**2)
        if output:
            print(f'At {chkpt.time}s: ({chk_x}, {chk_h})ft ==> Error = {chk_err:.8f}ft')
        return chk_err
    return float('inf')

summary = []

60.0 seconds to return to zero, launching at 1930.4430000000002fps with 30.0 degrees.  Range=100308.7607ft.


# SciPy

Here's a simple example to show what we're working with:

In [3]:
config = SciPyEngineConfigDict(
        cMinimumVelocity=0,
        cMinimumAltitude=-1,
        integration_method="LSODA",
)
calc = Calculator(config, engine='scipy_engine')
shot = Shot(ammo=Ammo(DragModel(bc=0.759, drag_table=TableG1), Velocity.FPS(launch_velocity_fps)), atmo=Vacuum())
shot.relative_angle = Angular.Degree(launch_angle_deg)
hit, e = must_fire(calc, shot, Distance.Feet(range_to_zero), extra_data=True, time_step=60.0)
check_error(hit, True)
hit.dataframe(True).drop(columns=['windage', 'slant_height', 'drop_adj', 'windage_adj', 'slant_distance', 'density_factor', 'drag', 'energy', 'ogw'])

INFO:py_balcalc:Loaded calculator from: py_ballisticcalc:SciPyIntegrationEngine (Class: <class 'py_ballisticcalc.engines.scipy_engine.SciPyIntegrationEngine'>)
DEBUG:py_balcalc:SciPy integration via LSODA done with 16 function calls.


ZERO_DOWN: 59.9999999999988s to (100308.7607134686, 1.3404160230204362e-12).
At 59.9999999999988s: (100308.7607134686, 1.3404160230204362e-12)ft ==> Error = 0.00000000ft


,time,distance,velocity,mach,height,angle,flag
0,0.000 s,0.00 ft,1930.4 ft/s,1.73 mach,0.00 ft,30.0000 °,ZERO_UP|RANGE
1,30.000 s,50154.38 ft,1671.8 ft/s,1.58 mach,14478.32 ft,0.0000 °,APEX
2,60.000 s,100308.76 ft,1930.4 ft/s,1.73 mach,0.00 ft,-30.0000 °,ZERO_DOWN|RANGE


In [4]:
def scipy_chk(timeit: bool = False, **kwargs):
    config = SciPyEngineConfigDict(
        cMinimumVelocity=0,
        cMinimumAltitude=-1,
        **kwargs,
    )
    calc = Calculator(config, engine='scipy_engine')
    shot = Shot(ammo=Ammo(DragModel(bc=0.759, drag_table=TableG1), Velocity.FPS(launch_velocity_fps)), atmo=Vacuum())
    shot.relative_angle = Angular.Degree(launch_angle_deg)
    t = calc.fire(shot, Distance.Feet(range_to_zero), extra_data=True, time_step=60.0)
    err = check_error(t, False)
    evals = calc.integration_step_count
    if timeit:
        speed = %timeit -o calc.fire(shot, Distance.Feet(range_to_zero), extra_data=True, time_step=60.0)
        return err, evals, speed.average
    return err, evals
logger.setLevel(logging.WARNING)
err, count, speed = scipy_chk(timeit=True, integration_method="LSODA")
print(f'Error={err:.8f}ft.  Integration steps: {calc.integration_step_count}.  Speed: {speed:.5f}s')

2.12 ms ± 469 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
Error=0.00000000ft.  Integration steps: 16.  Speed: 0.00212s


### Integration Methods

In [5]:
from py_ballisticcalc.engines.scipy_engine import INTEGRATION_METHOD
stdout = sys.stdout  # Capture stdout because timeit prints to it
sys.stdout = io.StringIO()
for method in get_args(INTEGRATION_METHOD):
    err, count, speed = scipy_chk(timeit=True, integration_method=method)
    summary.append(('SciPy', method, err, count, speed))
sys.stdout = stdout
pd.DataFrame(summary, columns=['Engine', 'Setting', 'Error (ft)', 'Integration Steps', 'Speed']).sort_values(by='Error (ft)', ascending=True)

,Engine,Setting,Error (ft),Integration Steps,Speed
0,SciPy,RK23,5.456968e-12,23,0.002864
3,SciPy,Radau,1.466516e-11,53,0.005803
1,SciPy,RK45,5.823608e-11,32,0.003373
2,SciPy,DOP853,1.891836e-10,77,0.005200
5,SciPy,LSODA,1.993613e-09,16,0.001774
4,SciPy,BDF,1.884764e-07,58,0.007934


### Tolerance

In [6]:
results = []
method = 'LSODA'
# Run with rtol as limiting factor
rtol = 1e-1
atol = 1e-13
while rtol > 1e-10:
    rtol /= 10.0
    err, count = scipy_chk(integration_method=method, relative_tolerance=rtol, absolute_tolerance=atol)
    results.append((atol, rtol, err, count))
# Run with atol as limiting factor
atol = 1e-1
rtol = 1e-13
while atol > 1e-10:
    atol /= 10.0
    err, count = scipy_chk(integration_method=method, relative_tolerance=rtol, absolute_tolerance=atol)
    results.append((atol, rtol, err, count))
pd.DataFrame(results, columns=['Absolute Tolerance', 'Relative Tolerance', 'Error (ft)', 'Integration Steps']).sort_values(by='Error (ft)', ascending=False).sort_values(by='Integration Steps', ascending=True)

,Absolute Tolerance,Relative Tolerance,Error (ft),Integration Steps
14,1.000000e-06,1.000000e-13,4.514004e-08,14
12,1.000000e-04,1.000000e-13,4.512549e-08,14
10,1.000000e-02,1.000000e-13,4.512549e-08,14
11,1.000000e-03,1.000000e-13,4.511094e-08,14
15,1.000000e-07,1.000000e-13,4.412141e-08,14
13,1.000000e-05,1.000000e-13,4.512549e-08,15
16,1.000000e-08,1.000000e-13,1.388253e-08,15
17,1.000000e-09,1.000000e-13,1.891749e-10,18
18,1.000000e-10,1.000000e-13,2.925916e-11,19
19,1.000000e-11,1.000000e-13,1.684450e-11,21


### Points computed

In [7]:
config = SciPyEngineConfigDict(
    cMinimumVelocity=0,
    cMinimumAltitude=-1,
    relative_tolerance=1e-13,
    absolute_tolerance=1e-8,
    integration_method="LSODA",
)
calc = Calculator(config, engine='scipy_engine')
shot = Shot(ammo=Ammo(DragModel(bc=0.759, drag_table=TableG1), Velocity.FPS(launch_velocity_fps)), atmo=Vacuum())
shot.relative_angle = Angular.Degree(launch_angle_deg)
hit, e = must_fire(calc, shot, Distance.Feet(range_to_zero), extra_data=True, time_step=60.0)
calc.eval_points

[0.0,
 1.5753354422458785e-05,
 1.5753354422458785e-05,
 3.150670884491757e-05,
 3.150670884491757e-05,
 0.15756505093343276,
 0.31509859515802063,
 0.4726321393826085,
 2.047967581628487,
 3.6233030238743655,
 5.198638466120244,
 20.951992888579028,
 36.70534731103781,
 52.458701733496596,
 68.21205615595538]

# Other engines

In [8]:
def vacuum_chk(engine_name: str, timeit: bool = False, calcStepSizeFeet: float = 1.0):
    config = BaseEngineConfigDict(
        cMinimumVelocity=0,
        cMinimumAltitude=-1,
        cMaxCalcStepSizeFeet=calcStepSizeFeet,
    )
    calc = Calculator(config, engine=engine_name)
    shot = Shot(ammo=Ammo(DragModel(bc=0.759, drag_table=TableG1), Velocity.FPS(launch_velocity_fps)), atmo=Vacuum())
    shot.relative_angle = Angular.Degree(launch_angle_deg)
    hit, e = must_fire(calc, shot, Distance.Feet(range_to_zero), extra_data=True, time_step=60.0)
    err = check_error(hit, False)
    evals = calc.integration_step_count
    if timeit:
        speed = %timeit -o must_fire(calc, shot, Distance.Feet(range_to_zero), extra_data=True, time_step=60.0)
        return err, evals, speed.average
    return err, evals

## Verlet steps

In [9]:
step = 0.5
while step <= 1000.0:
    err, count, speed = vacuum_chk('verlet_engine', timeit=True, calcStepSizeFeet=step)
    #print(f'Error={err:.8f}ft.  Integration steps: {count}')
    summary.append(('Verlet', step, err, count, speed))
    step *= 4.0
df = pd.DataFrame(summary, columns=['Engine', 'Setting', 'Error (ft)', 'Integration Steps', 'Speed'])
df[df.Engine == 'Verlet']

2.91 s ± 39 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
815 ms ± 33.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
271 ms ± 85.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
55.7 ms ± 7.96 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
13.8 ms ± 1.02 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)
4.28 ms ± 618 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


,Engine,Setting,Error (ft),Integration Steps,Speed
6,Verlet,0.5,0.250000,211265,2.908653
7,Verlet,2.0,0.999997,52817,0.815317
8,Verlet,8.0,3.999983,13206,0.271442
9,Verlet,32.0,15.999929,3303,0.055688
10,Verlet,128.0,63.986863,827,0.013817
11,Verlet,512.0,255.755962,208,0.004282


## Euler steps

In [10]:
step = 0.25
while step <= 100.0:
    try:
        err, count, speed = vacuum_chk('euler_engine', timeit=True, calcStepSizeFeet=step)
    except Exception as e:
        break
    #print(f'Error={err:.8f}ft.  Integration steps: {count}')
    summary.append(('Euler', step, err, count, speed))
    step *= 4.0
df = pd.DataFrame(summary, columns=['Engine', 'Setting', 'Error (ft)', 'Integration Steps', 'Speed'])
df[df.Engine == 'Euler']

9.69 s ± 512 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
2.37 s ± 75.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
538 ms ± 13.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
135 ms ± 5.01 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


,Engine,Setting,Error (ft),Integration Steps,Speed
12,Euler,0.25,0.068663,845053,9.694433
13,Euler,1.0,0.274653,211265,2.369243
14,Euler,4.0,1.098616,52816,0.538342
15,Euler,16.0,4.394466,13204,0.135375


## RK4 steps

In [ ]:
step = 0.5
while step <= 5000.0:
    err, count, speed = vacuum_chk('rk4_engine', timeit=True, calcStepSizeFeet=step)
    #print(f'Error={err:.8f}ft.  Integration steps: {count}')
    summary.append(('RK4', step, err, count, speed))
    step *= 4.0
df = pd.DataFrame(summary, columns=['Engine', 'Setting', 'Error (ft)', 'Integration Steps', 'Speed'])
df[df.Engine == 'RK4']

1.71 s ± 26 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
433 ms ± 31.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
110 ms ± 13.4 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
25.8 ms ± 675 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)
6.6 ms ± 159 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)
1.93 ms ± 69.9 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
595 μs ± 12.8 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
308 μs ± 9.39 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


,Engine,Setting,Error (ft),Integration Steps,Speed
16,RK4,0.5,0.000003,52817,1.710777
17,RK4,2.0,0.000017,13205,0.433014
18,RK4,8.0,0.000071,3302,0.110309
19,RK4,32.0,0.013124,826,0.025849
20,RK4,128.0,0.243265,207,0.006602
21,RK4,512.0,4.455340,52,0.001929
22,RK4,2048.0,27.466512,13,0.000595
23,RK4,8192.0,954.227487,4,0.000308


# Summary
The "error" is effectively the vertical error at the distance where the trajectory should be at height=0.

In [14]:
display(df.style.format({'Error (ft)': '{:.12f}'}))

,Engine,Setting,Error (ft),Integration Steps,Speed
0,SciPy,RK23,0.000000000005,23,0.002864
1,SciPy,RK45,0.000000000058,32,0.003373
2,SciPy,DOP853,0.000000000189,77,0.005200
3,SciPy,Radau,0.000000000015,53,0.005803
4,SciPy,BDF,0.000000188476,58,0.007934
5,SciPy,LSODA,0.000000001994,16,0.001774
6,Verlet,0.500000,0.249999957194,211265,2.908653
7,Verlet,2.000000,0.999996673655,52817,0.815317
8,Verlet,8.000000,3.999983183112,13206,0.271442
9,Verlet,32.000000,15.999929420339,3303,0.055688
